앞 장에서 신경망 학습에 대해서 설명했다. 신경망의 가중치 매개변수의 기울기는 수치 미분을 사용해 구했었다. 수치 미분은 단순하고 구현하기도 쉽지만 계산 시간이 오래 걸린다는게 단점이다. 이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 '오차역전파법'을 배워보자.

오차역전파를 이해하는 두 가지 방법
- 수식을 이용
- **계산 그래프 이용**


출처: https://excelsior-cjh.tistory.com/171 [EXCELSIOR]

## 5.1 계산 그래프

**계산 그래프(computational graph)**는 계산 과정을 그래프로 나타낸 것이며, 노드(node)와 엣지(edge)로 표현된다. 노드는 연산을 정의하며, 엣지는 데이터가 흘러가는 방향을 나타낸다.


### 5.1.1 계산 그래프 방법

다음의 예를 계산 그래프 방법으로 풀어보도록 하자.

> 현빈 군은 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비세가 10%일 때 지불 금액을 구하라.

**[그림 5-3] 계산 그래프로 풀어본 문제 답**
<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-2.png?raw=true">


1. 계산 그래프를 구성한다.
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. → 순전파(forward propagation)

### 5.1.2 국소적 계산

계산 그래프의 특징은 '국소적 계산'을 통해 최종 결과를 얻는 것이다. 즉, '자신과 직접 관계된' 범위 내에서만 계산이 이루어 진다. 

> 슈퍼마켓에서 사과 2개를 포함한 여러 식품을 구입하는 경우. 아래와 같은 그림으로 나타낼 수 있다.

**[그림 5-4] 사과 2개를 포함해 여러 식품을 구입하는 예**
<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-4.png?raw=true">


여기에서의 핵심은 각 노드에서의 계산은 국소적 계산이라는 점이다. 가령 사과와 그 외의 물품 값을 더하는 계산(2000 + 200 -> 4200)은 4000이라는 숫자가 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 뜻이다.   

#### 즉,각 노드는 자신과 관련한 계산 외에는 아무것도 신경 쓸 게 없다.

### 5.1.3 왜 계산 그래프로 푸는가?

계산그래프의 장점은 다음과 같다.

1. 국소적 계산을 통해 각 노드의 계산에 집중하여 문제를 단순화할 수 있다.

2. **역전파를 통해 '미분'을 효율적으로 계산할 수 있다.**

> '사과 가격이 오르면 최종 금액에 어떠한 영향을 주는가'에 대해서 **사과 가격에 대한 지불 금액의 미분**을 구해 계산할 수 있다. 사과의 값을 $x$, 지불 금액을 $L$라 했을 때, $\frac{\partial L}{\partial x}$를 구하는 것이다. 이러한 미분 값은 사과 값($x$)가 '아주 조금'올랐을 때 지불 금액($L$)이 얼마나 증가하는지를 나타낸다.

**[그림 5-5] 역전파에 의한 미분 값의 전달**

<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-5.png?raw=true">

이 전파는 '국소적 미분'을 전달하고 그 미분 값은 화살표의 아래에 적는다. 이 예에서 역전파는 오른쪽에서 왼쪽으로 `1-> 1.1 -> 2.2`순으로 미분값을 전달한다. 이 결과 '사과 가격에 대한 지불 금액의 미분'값은 2.2라 할 수 있다. 사과가 1원 오르면 최종 금액은 2.2원 오른다는 뜻이다.(정확히는 사과 값이 아주 조금 오르면 최종 금액은 그 아주 작은 값의 2.2배만큼 오른다는 것이다.)

## 5.2 연쇄법칙

지금까지 해온 계산 그래프의 순전파는 계산 결과를 왼쪽에서 오른쪽으로 전달했었다. 이 순서는 평소 하는 방식이라 어려움이 없었지만, 역전파는 '국소적인 미분'을 순방향과는 반대인 오른쪽에서 왼쪽으로 전달한다.(지금 봐도 당황)..
또한, 이 국소적인 미분을 전달하는 원리는 **연쇄법칙**에 따른 것이다. 

이번 절에서는 연쇄법칙을 설명하고 그것이 계산 그래프 상의 역전파와 같다는 사실을 밝힌다.

### 5.2.1 계산 그래프의 역전파

먼저, 역전파 계산 예제로 $y = f(x)$의 역전파를 계산해보자. 

**[그림 5-6] 계산 그래프의 역전파 : 순방향과는 반대 방향으로 국소적 미분을 곱한다.**

<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-6.png?raw=true">

위의 그림에서 처럼 역전파 계산 순서는 신호 $E$에 노드$f(x)$의 국소적 미분 ($\frac{\partial y}{\partial x}$)을 곱한 후 엣지(edge)를 통해 다음 노드로 전달하는 것이다. 여기서 국소적 미분은 순전파 때의 $y = f(x)$에 대한 미분을 구하는 것이고, 이것은 $x$에 대한 $y$의 미분 ($\frac{\partial L}{\partial x}$)을 구한다는 의미이다.


### 5.2.2 연쇄법칙이란?

합성 함수 이야기부터 시작해야 합니다. **합성함수**란 여러 함수로 구성된 함수이다.

**예제**

$$
z = t^2
$$

$$
t = x + y
$$

연쇄법칙은 합성 함수의 미분에 대한 성질이며, 다음과 같이 정의된다.

> **합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.**

예제 : $\frac{\partial z}{\partial x}$ ($x$에 대한 $z$의 미분)은 $\frac{\partial z}{\partial t}$ ($t$에 대한 $z$의 미분)과 $\frac{\partial t}{\partial x}$ ($x$에 대한 $t$의 미분)의 곱으로 나타낼 수 있다.

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

$$
\frac{\partial z}{\partial t} = 2t
$$

$$
\frac{\partial t}{\partial x} = 1
$$

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t \cdot 1 = 2(x+y)
$$


### 5.2.3 연쇄법칙과 계산 그래프

위의 식의 연쇄법칙 계산을 계산 그래프로 나타내보자.

**[그림 5-7] 계산 그래프 : 순전파와는 반대 방향으로 국소적 미분을 곱하여 전달한다.**

<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-7.png?raw=true">

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

덧셈 노드의 역전파는 입력값을 그대로 흘려보낸다. 이를 보고 gradient distributor라고 하기도 한다.

$$
z = x + y
$$

$$
\frac{\partial z}{\partial x} = 1
$$

$$
\frac{\partial z}{\partial y} = 1
$$

**[그림 5-9] 덧셈 노드의 역전파 : 왼쪽이 순전파, 오른쪽이 역전파다. 덧셈 노드의 역전파는 입력 값을 그대로 흘려보낸다.**

<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-9.png?raw=true">

### 5.3.2 곱셈 노드의 역전파

곱셈 노드의 역전파는 입력값의 위치를 서로 바꾼 다음 곱해서 흘려보낸다. 이를 보고 gradient switcher라고 부르기도 한다.

$$
z = xy
$$

$$
\frac{\partial z}{\partial x} = y
$$

$$
\frac{\partial z}{\partial y} = x
$$

**[그림 5-12] 곱셈 노드의 역전파 : 왼쪽이 순전파.**

<img src="https://github.com/DeepHaeJoong/DeepLearningFromScratch_I/blob/master/PNG/Figure%205-12.png?raw=true">

곱셈의 역전파는 순방향 입력 신호의 값이 필요합니다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장한다.

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

모든 계층은 **forward()**와 **backward()**라는 공통의 메서드(인터페이스)를 갖도록 구현합니다. forward()는 순전파, backward()는 역전파를 처리한다.

**[그림 5-16] 사과 2개 구입**

In [4]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

순전파를 다음과 같이 구현해보자.

In [6]:
apple = 100
apple_num = 2
tax = 1.1

# 게층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


역전파를 다음과 같이 구현해보자.

In [13]:
# 역전파 
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print('dapple :',dapple, '\ndapple_num :',dapple_num, '\ndtax : ', dtax)

dapple : 2.2 
dapple_num : 110.00000000000001 
dtax :  200


### 5.4.2 덧셈 계층

이어서 덧셈 노드인 덧셈 계층을 구현해보자.

**[그림 5-17] 사과 2개와 귤 3개 구입**

In [16]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [20]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print('%d' % price)
print("%d, %.1f, %.1f, %d, %d" % (dapple_num, dapple, dorange, dorange_num, dtax))

715
110, 2.2, 3.3, 165, 650


- 필요 계층을 만들어 순전파 메서드인 forward()를 적절한 순서로 호출한다.
- 그런 다음 순전파와 반대 순서로 역전파 매서드인 forward()를 호출하면 원하는 미분이 나온다.

이처럼 계산 그래프에서 계층을 쉽게 구현할 수 있으며, 이를 사용해 복잡한 미분도 계산할 수 있다.

## 5.5 활성화 함수 계층 구현하기

- ReLU
- Sigmoid

### 5.5.1 ReLU 계층

- **수식**


$$
y = \begin{cases} 
     x \quad (x > 0) \\
     0 \quad (x <= 0) \\
    \end{cases}
$$

- **미분**


$$
\frac{\partial y}{\partial x} = \begin{cases} 
     1 \quad (x > 0) \\
     0 \quad (x <= 0) \\
    \end{cases}
$$

**[그림 5-18] ReLU 계층의 계산 그래프**

In [44]:
# common/layers.py
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0) # mask는 True/False로 구성된 넘파이 배열
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [47]:
x = np.array([[1., -.5], 
              [-2., 3.]])
print('x:\n', x)

mask = (x <= 0)
print('mask:\n', mask)

x:
 [[ 1.  -0.5]
 [-2.   3. ]]
mask:
 [[False  True]
 [ True False]]


> 순전파 때의 입력 값이 0이하면 역전파 때의 값은 0이 돼야 한다. 그래서 역전파 때는 순전파 때 만들어둔 mask를 사용해 mask의 원소가 True인 곳에는 상류에서 전파된 dout을 0으로 설정한다.

### 5.5.2 Sigmoid 계층

- **수식**


$$
y = \frac{1}{1+exp(-x)}
$$



**[그림 5-19] Sigmoid 계층의 계산 그래프는 식을 순차적으로 전개함**

#### 1단계

**`/`** 노드, 즉 $y = \frac{1}{x}$ 을 미분하면 다음과 같다.
$$
\frac{\partial y}{\partial x} = -\frac{1}{x^2}
$$

$$
\frac{\partial y}{\partial x} = -y^2
$$



#### 2단계
**`+`** 노드는 상류의 값을 여과 없이 하류로 내보내는 게 전부이다. 계산 그래프에서는 다음과 같다.

#### 3단계
**`exp`** 노드는 $y=exp(x)$의 연산을 수행하며, 그 미분은 다음과 같다.

$$
\frac{\partial y}{\partial x} = exp(x)
$$

계산 그래프에서는 상류의 값에 순전파 때의 출력(이 예제에서는 $y=exp(-x)$)을 곱해 하류로 전파한다.

#### 4단계
**`x`** 노드는 순전파 때의 값을 '서로 바꿔'곱하게 된다. 이 예제에서는 -1을 곱하면 된다.

$$
\frac{\partial L}{\partial y} y^2 exp(-x) = \frac{\partial L}{\partial y}y(1-y)
$$

In [50]:
# common/layers
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        self.out
        
        return dx

## 5.6 Affine/Softmax 계층 구현하기

### 5.5.6 Affine 계층